In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import math
import json
import pickle
import tensorflow.python.keras as keras
from keras.layers import Input, Layer, LeakyReLU, BatchNormalization, Conv2D, MaxPooling2D, UpSampling2D, Concatenate, Add, Lambda
from keras.models import Model, load_model, model_from_json, clone_model
from keras.optimizers import SGD, Adam
import keras.backend as K
from PIL import Image
import os
K.clear_session()
'''
# file h5 dibuat dengan convert.py dari https://github.com/awe777/keras-yolo3
model0 = load_model("yolov3_tiny.h5")   # dibuat dari kode sebelum fork
model1 = load_model("yolov3_tiny_.h5")  # dibuat dari kode setelah diubah di fork pribadi
print(model0.to_json() == model1.to_json())
print(model.summary())
# '''
print()

Using TensorFlow backend.


In [2]:
def roundingAlgo(x):
    return K.round(x)

class RoundQinf_4(Layer):
    def __init__(self, **kwargs):
        super(RoundQinf_4, self).__init__(**kwargs)
        self.trainable = False
    def build(self, input_shape):
        super(RoundQinf_4, self).build(input_shape)
    def call(self, X):
        return roundingAlgo(X * 16) / 16.0
    def get_config(self):
        base_config = super(RoundQinf_4, self).get_config()
        return dict(list(base_config.items()))
class RoundOverflowQ7_12(Layer):
    def __init__(self, **kwargs):
        super(RoundOverflowQ7_12, self).__init__(**kwargs)
        self.trainable = False
    def build(self, input_shape):
        super(RoundOverflowQ7_12, self).build(input_shape)
    def call(self, X):
        return (((roundingAlgo(X * 4096) + 524288) % 1048576) - 524288) / 4096.0
    def get_config(self):
        base_config = super(RoundOverflowQ7_12, self).get_config()
        return dict(list(base_config.items()))
class RoundQx_8minusx(Layer):
    def __init__(self, **kwargs):
        super(RoundQx_8minusx, self).__init__(**kwargs)
        self.trainable = False
    def build(self, input_shape):
        super(RoundQx_8minusx, self).build(input_shape)
    def call(self, X):
        pow_2_to_expplus8 = 2 ** (8 + (K.log(K.abs(X)) / math.log(2)) // 1)
        return roundingAlgo(X * pow_2_to_expplus8) / (pow_2_to_expplus8 * 1.0)
    def get_config(self):
        base_config = super(RoundQx_8minusx, self).get_config()
        return dict(list(base_config.items()))
class RoundOverflowQ3_4(Layer):
    def __init__(self, **kwargs):
        super(RoundOverflowQ3_4, self).__init__(**kwargs)
        self.trainable = False
    def build(self, input_shape):
        super(RoundOverflowQ3_4, self).build(input_shape)
    def call(self, X):
        return (((roundingAlgo(X * 16) + 128) % 256) - 128) / 16.0
    def get_config(self):
        base_config = super(RoundOverflowQ3_4, self).get_config()
        return dict(list(base_config.items()))
class RoundOverflowQ7_0(Layer):
    def __init__(self, **kwargs):
        super(RoundOverflowQ7_0, self).__init__(**kwargs)
        self.trainable = False
    def build(self, input_shape):
        super(RoundOverflowQ7_0, self).build(input_shape)
    def call(self, X):
        return ((roundingAlgo(X) + 128) % 256) - 128
    def get_config(self):
        base_config = super(RoundOverflowQ7_0, self).get_config()
        return dict(list(base_config.items()))
class Identity(Layer):
    def __init__(self, **kwargs):
        super(Identity, self).__init__(**kwargs)
        self.trainable = False
    def call(self, X):
        return X
    def get_config(self):
        base_config = super(Identity, self).get_config()
        return dict(list(base_config.items()))
class IdentityFinalLayer(Layer):
    def __init__(self, **kwargs):
        super(IdentityFinalLayer, self).__init__(**kwargs)
        self.trainable = False
    def call(self, X):
        return X
    def get_config(self):
        base_config = super(IdentityFinalLayer, self).get_config()
        return dict(list(base_config.items()))

print("Custom layer classes successfully defined")

Custom layer classes successfully defined


In [3]:
customObjects = {
    'Identity': Identity,
    'RoundQinf_4': RoundQinf_4,
    'RoundOverflowQ3_4': RoundOverflowQ3_4,
    'RoundQx_8minusx': RoundQx_8minusx,
    'RoundOverflowQ7_12': RoundOverflowQ7_12,
    'RoundOverflowQ7_0': RoundOverflowQ7_0,
    'IdentityFinalLayer': IdentityFinalLayer
}

In [4]:
def prepare_image(image_dir, input_shape):
    # from PIL import Image
    # import numpy as np
    image_data = 0
    h, w = input_shape
    try:
        image = Image.open(image_dir)
        iw, ih = image.size
        scale = min(w/iw, h/ih)
        nw = int(iw*scale)
        nh = int(ih*scale)
        dx = (w-nw)//2
        dy = (h-nh)//2
        image = image.resize((nw,nh), Image.BICUBIC)
        new_image = Image.new('RGB', (w,h), (128,128,128))
        new_image.paste(image, (dx, dy))
        image_data = np.array(new_image)/255.
        return image_data
    except:
        return None
# TO DO: implement yolo_head in order to get human-understandable result from model output

def get_anchors(anchors_path):
    '''loads the anchors from a file'''
    with open(anchors_path) as f:
        anchors = f.readline()
    anchors = [float(x) for x in anchors.split(',')]
    return np.array(anchors).reshape(-1, 2)

tinyYolo_anchors = get_anchors("../CNN-VLSI/tiny_yolo_anchors.txt")

def yolo_head_nonLoss(feats, anchors, num_classes, input_shape):
    """Convert final layer features to bounding box parameters."""
    num_anchors = len(anchors)
    # Reshape to batch, height, width, num_anchors, box_params.
    anchors_tensor = K.reshape(K.constant(anchors), [1, 1, 1, num_anchors, 2])

    grid_shape = K.shape(feats)[1:3] # height, width
    grid_y = K.tile(K.reshape(K.arange(0, stop=grid_shape[0]), [-1, 1, 1, 1]), [1, grid_shape[1], 1, 1])
    grid_x = K.tile(K.reshape(K.arange(0, stop=grid_shape[1]), [1, -1, 1, 1]), [grid_shape[0], 1, 1, 1])
    grid = K.concatenate([grid_x, grid_y])
    grid = K.cast(grid, 'float32') # K.cast(grid, K.dtype(feats))

    feats = K.reshape(
        feats, [-1, grid_shape[0], grid_shape[1], num_anchors, num_classes + 5])

    # Adjust preditions to each spatial grid point and anchor size.
    # edited by instructions in https://stackoverflow.com/questions/57558476/training-a-keras-model-yields-multiple-optimizer-errors
    box_xy = (K.sigmoid(feats[..., :2]) + grid) / K.cast(grid_shape[...,::-1], K.dtype(feats))
    box_wh = K.exp(feats[..., 2:4]) * anchors_tensor / K.cast(input_shape[...,::-1], K.dtype(feats))
    box_confidence = K.sigmoid(feats[..., 4:5])
    box_class_probs = K.sigmoid(feats[..., 5:])

    return box_xy, box_wh, box_confidence, box_class_probs

def yolo_correct_boxes(box_xy, box_wh, input_shape, image_shape):
    '''Get corrected boxes'''
    box_yx = box_xy[..., ::-1]
    box_hw = box_wh[..., ::-1]
    input_shape = K.cast(input_shape, K.dtype(box_yx))
    image_shape = K.cast(image_shape, K.dtype(box_yx))
    new_shape = K.round(image_shape * K.min(input_shape/image_shape))
    offset = (input_shape-new_shape)/2./input_shape
    scale = input_shape/new_shape
    box_yx = (box_yx - offset) * scale
    box_hw *= scale

    box_mins = box_yx - (box_hw / 2.)
    box_maxes = box_yx + (box_hw / 2.)
    boxes =  K.concatenate([
        box_mins[..., 0:1],  # y_min
        box_mins[..., 1:2],  # x_min
        box_maxes[..., 0:1],  # y_max
        box_maxes[..., 1:2]  # x_max
    ])

    # Scale boxes back to original image shape.
    boxes *= K.concatenate([image_shape, image_shape])
    return boxes

def yolo_boxes_and_scores(feats, anchors, num_classes, input_shape, image_shape):
    '''Process Conv layer output'''
    box_xy, box_wh, box_confidence, box_class_probs = yolo_head_nonLoss(feats,
        anchors, num_classes, input_shape)
    boxes = yolo_correct_boxes(box_xy, box_wh, input_shape, image_shape)
    boxes = K.reshape(boxes, [-1, 4])
    box_scores = box_confidence * box_class_probs
    box_scores = K.reshape(box_scores, [-1, num_classes])
    return boxes, box_scores

def yolo_eval(yolo_outputs,
              anchors,
              num_classes,
              image_shape,
              max_boxes=20,
              score_threshold=.6,
              iou_threshold=.5):
    """Evaluate YOLO model on given input and return filtered boxes."""
    num_layers = len(yolo_outputs)
    anchor_mask = [[6,7,8], [3,4,5], [0,1,2]] if num_layers==3 else [[3,4,5], [1,2,3]] # default setting
    input_shape = K.shape(yolo_outputs[0])[1:3] * 32
    boxes = []
    box_scores = []
    for l in range(num_layers):
        _boxes, _box_scores = yolo_boxes_and_scores(yolo_outputs[l],
            anchors[anchor_mask[l]], num_classes, input_shape, image_shape)
        boxes.append(_boxes)
        box_scores.append(_box_scores)
    boxes = K.concatenate(boxes, axis=0)
    box_scores = K.concatenate(box_scores, axis=0)
#     print("Box tensor:", boxes.shape)
#     print(K.get_value(boxes))
#     print("Box score tensor:", box_scores.shape)
#     print(K.get_value(box_scores))
#     print("class index of maximum prob value:", K.argmax(box_scores, axis=1).shape)
#     print(K.get_value(K.argmax(box_scores, axis=1)))
#     print("maximum prob value: ",K.max(box_scores, axis=1).shape)
#     print(K.get_value(K.max(box_scores, axis=1)))
#     print("maximum prob mask: ", (K.max(box_scores, axis=1) >= score_threshold).shape)
#     print(K.get_value(K.max(box_scores, axis=1) >= score_threshold)) # 0.027))
#     print()
#     print("index of highest maximum prob value:",K.get_value(K.argmax(K.max(box_scores, axis=1))))
#     print("highest maximum prob value:", K.get_value(K.max(box_scores)))
#     print("class index:", K.get_value(K.argmax(box_scores, axis=1))[K.get_value(K.argmax(K.max(box_scores, axis=1)))])
#     print("raw coordinates:", K.get_value(boxes)[K.get_value(K.argmax(K.max(box_scores, axis=1)))])
#     print("class index:", K.get_value(box_scores)[K.get_value(K.argmax(K.max(box_scores, axis=1)))])
    boxes_dummy = []
    scores_dummy = []
    classes_dummy = []
    scoremask_dummy = K.get_value(K.max(box_scores, axis=1) >= score_threshold)
    for i in range(scoremask_dummy.size):
        if scoremask_dummy[i]:
            boxes_dummy.append(K.get_value(boxes)[i].tolist())
            scores_dummy.append(K.get_value(K.max(box_scores, axis=1))[i].tolist())
            classes_dummy.append(K.get_value(K.argmax(box_scores, axis=1))[i].tolist())
    ''' # toggle to utilize either hacky solution with working classes output or IOU result-cutting solution with non-working classes output
    boxes_ = []
    scores_ = []
    classes_ = []
    mask = box_scores >= score_threshold
    max_boxes_tensor = K.constant(max_boxes, dtype='int32')
    for c in range(num_classes):
        # TODO: use keras backend instead of tf.
        class_boxes = tf.boolean_mask(boxes, mask[:, c])
        class_box_scores = tf.boolean_mask(box_scores[:, c], mask[:, c])
        nms_index = tf.image.non_max_suppression(
            class_boxes, class_box_scores, max_boxes_tensor, iou_threshold=iou_threshold)
        class_boxes = K.gather(class_boxes, nms_index)
        class_box_scores = K.gather(class_box_scores, nms_index)
        classes = K.ones_like(class_box_scores, 'int32') * c
        boxes_.append(class_boxes)
        scores_.append(class_box_scores)
        classes_.append(classes)
    boxes_ = K.concatenate(boxes_, axis=0)
    scores_ = K.concatenate(scores_, axis=0)
    classes_ = K.concatenate(classes_, axis=0)
    '''
    boxes_ = boxes_dummy
    scores_ = scores_dummy
    classes_ = classes_dummy
    '''
    # '''

    return boxes_, scores_, classes_

In [5]:
loadedModel = None
try:
    # '''
    with open("./saved_models/model_5_inferenceModel.json", "r") as jsonFile:
        jsonRead = jsonFile.read()
        # NOTE: this *will* take time, may cause OOM warnings
        loadedModel = model_from_json(jsonRead, custom_objects=customObjects)
        copyLoadModel = clone_model(loadedModel)
        loadedModel = copyLoadModel
        del copyLoadModel
    # '''
    print("Model loading successful")
    ''' # toggle comment to utilize by-name model (h5 and json are based on same model)
    loadedModel.load_weights("./saved_models/model_0_trainModel.h5", by_name=True, skip_mismatch=True)
    ''' # or not (e.g. h5 from model_0, json from other)
    loadedModel.load_weights("./saved_models/model_0_trainModel.h5", by_name=False)    
    # '''   
    print("Model weight loading attempt successful")
except Exception as e:
    print("Failed to load model/weight data:", e)

Model loading successful
Model weight loading attempt successful


In [6]:
if loadedModel is not None:
#     image_directory = "./test2017/000000000202.jpg"
#     image_directory = "./test2017/000000001371.jpg"
#     image_directory = "./test2017/000000000647.jpg"
#     image_directory = "./train2017/000000376608.jpg"
    image_directory = "./train2017/000000000036.jpg"
    image_data = []
    prepImage = prepare_image(image_directory, (448,448))
    print(prepImage.shape)
    image_data = np.expand_dims(prepImage,axis=0)
    processed_model_output = yolo_eval(
        yolo_outputs = loadedModel.predict(
            x = image_data,
            verbose=1,
            steps=1
        ),
        anchors=tinyYolo_anchors,
        num_classes=80,
        image_shape=(448,448),
        max_boxes=20,
        score_threshold=0,
        iou_threshold=0.5
    )
    print("Found", len(processed_model_output[0]), "objects")
    for i in range(len(processed_model_output[0])):
        print("Obj", i)
        print("Top, Left, Bottom, Right: ", processed_model_output[0][i])
        print("Prob:", processed_model_output[1][i])
        print("Class:", processed_model_output[2][i])
print("Completed inference")

(448, 448, 3)
1/1 [==============================] - 0s 415ms/step
Found 2940 objects
Obj 0
Top, Left, Bottom, Right:  [-22595.650390625, -11.38522720336914, 22654.06640625, 42.611942291259766]
Prob: 0.06235333904623985
Class: 41
Obj 1
Top, Left, Bottom, Right:  [-3836.61865234375, -43425.49609375, 3887.974609375, 43457.34375]
Prob: 0.021978724747896194
Class: 38
Obj 2
Top, Left, Bottom, Right:  [-134.23411560058594, -10048.0498046875, 161.29730224609375, 10092.0048828125]
Prob: 0.04980255290865898
Class: 70
Obj 3
Top, Left, Bottom, Right:  [-55.23031997680664, -37.95111083984375, 85.38471984863281, 145.88217163085938]
Prob: 0.07293238490819931
Class: 38
Obj 4
Top, Left, Bottom, Right:  [-11148.5361328125, -332835.40625, 11208.771484375, 332930.125]
Prob: 0.07555445283651352
Class: 72
Obj 5
Top, Left, Bottom, Right:  [-169.0238037109375, -10007.587890625, 196.0869903564453, 10132.466796875]
Prob: 0.06411249935626984
Class: 68
Obj 6
Top, Left, Bottom, Right:  [-21.61220932006836, 10.095

Obj 493
Top, Left, Bottom, Right:  [286.6483459472656, 282.689208984375, 442.90966796875, 385.7937927246094]
Prob: 0.19807958602905273
Class: 0
Obj 494
Top, Left, Bottom, Right:  [260.82440185546875, 169.03848266601562, 467.0378723144531, 495.6069641113281]
Prob: 0.3079650104045868
Class: 60
Obj 495
Top, Left, Bottom, Right:  [297.235595703125, 332.5543212890625, 437.8506164550781, 396.69317626953125]
Prob: 0.06342481076717377
Class: 0
Obj 496
Top, Left, Bottom, Right:  [283.83294677734375, 318.0316467285156, 445.72503662109375, 411.1561279296875]
Prob: 0.14378799498081207
Class: 0
Obj 497
Top, Left, Bottom, Right:  [251.92373657226562, 201.03848266601562, 477.52191162109375, 527.60693359375]
Prob: 0.15617164969444275
Class: 60
Obj 498
Top, Left, Bottom, Right:  [297.7959899902344, 362.132080078125, 438.4110107421875, 434.06951904296875]
Prob: 0.0408330075442791
Class: 62
Obj 499
Top, Left, Bottom, Right:  [285.4383850097656, 342.89886474609375, 447.3304748535156, 451.3743896484375]
Pr

Class: 0
Obj 993
Top, Left, Bottom, Right:  [59.72562026977539, 353.0982666015625, 83.40804290771484, 396.958251953125]
Prob: 0.03676861897110939
Class: 56
Obj 994
Top, Left, Bottom, Right:  [45.021324157714844, 356.1951904296875, 101.99679565429688, 392.79986572265625]
Prob: 0.05244457349181175
Class: 56
Obj 995
Top, Left, Bottom, Right:  [35.934295654296875, 328.30364990234375, 107.92898559570312, 422.3111877441406]
Prob: 0.04304615780711174
Class: 0
Obj 996
Top, Left, Bottom, Right:  [53.80027770996094, 381.39306640625, 94.55036926269531, 402.4116516113281]
Prob: 0.06226702779531479
Class: 30
Obj 997
Top, Left, Bottom, Right:  [15.612733840942383, 379.34906005859375, 128.31500244140625, 409.04315185546875]
Prob: 0.03673618659377098
Class: 0
Obj 998
Top, Left, Bottom, Right:  [30.051475524902344, 324.29766845703125, 113.81180572509766, 465.35076904296875]
Prob: 0.03542601689696312
Class: 0
Obj 999
Top, Left, Bottom, Right:  [56.10984802246094, 391.1971740722656, 87.97804260253906, 42

Class: 0
Obj 1493
Top, Left, Bottom, Right:  [125.59713745117188, 307.0787353515625, 209.35748291015625, 379.86419677734375]
Prob: 0.03744509443640709
Class: 0
Obj 1494
Top, Left, Bottom, Right:  [147.6688995361328, 351.23077392578125, 188.4189910888672, 373.6869812011719]
Prob: 0.06462329626083374
Class: 0
Obj 1495
Top, Left, Bottom, Right:  [127.75048065185547, 342.63629150390625, 208.17726135253906, 377.11566162109375]
Prob: 0.05201936513185501
Class: 0
Obj 1496
Top, Left, Bottom, Right:  [109.35641479492188, 322.75213623046875, 225.90191650390625, 395.5375671386719]
Prob: 0.04691731557250023
Class: 0
Obj 1497
Top, Left, Bottom, Right:  [154.39353942871094, 361.6331481933594, 181.17686462402344, 389.8202209472656]
Prob: 0.036158885806798935
Class: 0
Obj 1498
Top, Left, Bottom, Right:  [127.75048065185547, 358.2882080078125, 208.17726135253906, 392.7676086425781]
Prob: 0.042275119572877884
Class: 0
Obj 1499
Top, Left, Bottom, Right:  [109.50758361816406, 339.9732971191406, 226.053070

Class: 0
Obj 2055
Top, Left, Bottom, Right:  [266.5013122558594, 204.91824340820312, 292.63232421875, 226.88644409179688]
Prob: 0.024562619626522064
Class: 0
Obj 2056
Top, Left, Bottom, Right:  [239.75048828125, 201.4623260498047, 320.17724609375, 234.92987060546875]
Prob: 0.04755045101046562
Class: 0
Obj 2057
Top, Left, Bottom, Right:  [222.5572509765625, 183.16152954101562, 339.102783203125, 254.4868927001953]
Prob: 0.05561831593513489
Class: 0
Obj 2058
Top, Left, Bottom, Right:  [267.6484069824219, 219.54788208007812, 291.92205810546875, 242.5086212158203]
Prob: 0.022676007822155952
Class: 0
Obj 2059
Top, Left, Bottom, Right:  [239.18328857421875, 212.7322998046875, 319.6100769042969, 248.2627716064453]
Prob: 0.04134894907474518
Class: 0
Obj 2060
Top, Left, Bottom, Right:  [221.6588897705078, 194.3338165283203, 338.2043762207031, 268.6091613769531]
Prob: 0.04682416096329689
Class: 0
Obj 2061
Top, Left, Bottom, Right:  [271.7025146484375, 233.8087921142578, 292.64813232421875, 261.99

Top, Left, Bottom, Right:  [336.18341064453125, 126.38967895507812, 415.6799011230469, 241.54197692871094]
Prob: 0.042115166783332825
Class: 0
Obj 2556
Top, Left, Bottom, Right:  [368.46270751953125, 186.68618774414062, 391.5739440917969, 214.87327575683594]
Prob: 0.029057394713163376
Class: 0
Obj 2557
Top, Left, Bottom, Right:  [345.2542419433594, 183.89378356933594, 413.1956481933594, 220.49844360351562]
Prob: 0.04626811295747757
Class: 0
Obj 2558
Top, Left, Bottom, Right:  [334.0514831542969, 154.28756713867188, 417.8117980957031, 248.29510498046875]
Prob: 0.0458662286400795
Class: 0
Obj 2559
Top, Left, Bottom, Right:  [363.3453674316406, 202.6862030029297, 388.22503662109375, 230.87326049804688]
Prob: 0.024723200127482414
Class: 0
Obj 2560
Top, Left, Bottom, Right:  [341.9931640625, 198.20480346679688, 409.9346008300781, 234.80946350097656]
Prob: 0.04031099006533623
Class: 0
Obj 2561
Top, Left, Bottom, Right:  [334.0514831542969, 176.49082946777344, 417.8117980957031, 255.440795898